In [1]:
import pandas as pd
import datetime
from waipawama import Finance
import pathlib
import datetime as dt
import redis

from fastcounting import helper

In [2]:
r = redis.Redis(**helper.rediscred, decode_responses=True)

In [3]:
lua_script = """
local result = 0.0
for l, atomic in ipairs(redis.call('SMEMBERS', KEYS[1]))
do result = result + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end
return result
"""

In [4]:
{**helper.rediscred}

{'host': 'localhost', 'port': 6379}

In [5]:
result = r.eval(lua_script, 1, 'account:atomic:3300.0')

In [6]:
result

0

In [45]:
r.hgetall('generalID:10')

{'date': '1483228800',
 'jourdat': '1487030400.0',
 'buchdat': '1487030400.0',
 'status': 'nan',
 'belegnr': 'AM171_15'}

In [46]:
r.hgetall('atomicID:2')

{'generalID': '1',
 'accountID': '4400.0',
 'text': '028-5828144-7937959',
 'amount': '550.0',
 'kontenseite': 'Haben'}

In [47]:
result

0

In [48]:
def read_lexware(month, name='report', nrows=None):
    """Read xlxs from the default folder for each year e.g. month=2018-13 or actual month."""
    p = pathlib.Path(f'{Finance().data}{month}')
    files = [file for file in p.iterdir() if file.parts[-1].lower().startswith(name)]
    if files:
        data = pd.read_table(files[0], sep='\t', engine='python', header=[0, 1], decimal=',', thousands='.', parse_dates=[2])
    return data

In [49]:
def clean_report(validate):
    better_columns = ['Konto', 'Name', 'Letzte Buchung', 'EB Soll', 'EB Haben', 'Summe Soll', 'Summe Haben', 'drop1', 'drop2', 'Saldo Soll', 'Saldo Haben']
    validate.columns = better_columns
    validate.set_index('Konto', inplace = True)
    validate.fillna(0, inplace=True)  # for full year data set they are equal to sum
    validate.drop(['drop1', 'drop2'], axis=1, inplace=True)
    return validate

In [50]:
# main
validate = read_lexware('2017-13')
validate = clean_report(validate)

In [51]:
validate['checksum'] =  validate['Saldo Haben'] - validate['Saldo Soll']

In [52]:
redis_sum_list = []
for konto in validate.index:
    redis_sum_list.append(r.eval(lua_script, 1, f'account:atomic:{float(konto)}'))

In [53]:
validate['test_redis'] = redis_sum_list
validate['test_redis'] = validate['test_redis'] / 100

In [57]:
validate

,Name,Letzte Buchung,EB Soll,EB Haben,Summe Soll,Summe Haben,Saldo Soll,Saldo Haben,checksum,test_redis
Konto,,,,,,,,,,
135,EDV-Software,27.11.,0.00,0.00,12.97,0.00,12.97,0.00,-12.97,-12.97
650,Büroeinrichtung,27.11.,854.84,0.00,524.97,0.00,1379.81,0.00,-1379.81,-1379.81
670,Geringwertige Wirtschaftsgüter,01.01.,128.40,0.00,0.00,0.00,128.40,0.00,-128.40,-128.40
675,Geringwertige Wirtschaftsgüter Sammelposten,31.12.,532.42,0.00,0.00,88.07,444.35,0.00,-444.35,-444.35
690,Sonstige Betriebs- und Geschäftsausstattung,31.12.,100.53,0.00,651.17,54.28,697.42,0.00,-697.42,-697.42
...,...,...,...,...,...,...,...,...,...,...
9008,Saldenvorträge Debitoren,01.01.,0.00,1.46,0.00,0.00,0.00,1.46,1.46,1.46
9009,Saldenvorträge Kreditoren,01.01.,0.00,5329.79,0.00,0.00,0.00,5329.79,5329.79,5329.79
10010,Kunden,27.04.,1.46,0.00,0.00,0.00,1.46,0.00,-1.46,-1.46


In [54]:
validate[validate['checksum'] != validate['test_redis']]

,Name,Letzte Buchung,EB Soll,EB Haben,Summe Soll,Summe Haben,Saldo Soll,Saldo Haben,checksum,test_redis
Konto,,,,,,,,,,
1200,Forderungen aus Lieferungen und Leistungen,27.04.,1.46,0.0,0.00,0.00,1.46,0.0,-1.46,0.0
3300,Verbindlichkeiten aus Lieferungen und Leistungen,31.12.,5329.79,0.0,445422.74,441193.95,9558.58,0.0,-9558.58,0.0


In [55]:
r.smembers('account:atomic:10007.0')

{'139'}

In [38]:
r.hgetall('atomicID:139')

{'generalID': '50',
 'accountID': '10007.0',
 'text': 'EB-Wert',
 'amount': '114.0',
 'kontenseite': 'Haben'}

In [39]:
m = 114

In [40]:
float(114)

114.0

In [41]:
float(int(float(m)))

114.0